In [1]:
import findspark
findspark.init()

In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, avg, col, concat, desc, explode, lit, min, max, split
from pyspark.sql.types import IntegerType

from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import CountVectorizer, IDF, Normalizer, PCA, RegexTokenizer, StandardScaler, StopWordsRemover, VectorAssembler, StringIndexer, MinMaxScaler, PCA
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

import re

In [6]:
data_dir = 'C:/Users/John/PycharmProjects/customer-attrition/data/'

In [7]:
spark = SparkSession.builder.appName("spark machine learning").getOrCreate()

In [8]:
df = spark.read.json(data_dir + 'Train_onetag_small.json')
df.persist()

DataFrame[Body: string, Id: bigint, Tags: string, Title: string, oneTag: string]

In [6]:
df.head()

Row(Body="<p>I'd like to check if an uploaded file is an image file (e.g png, jpg, jpeg, gif, bmp) or another file. The problem is that I'm using Uploadify to upload the files, which changes the mime type and gives a 'text/octal' or something as the mime type, no matter which file type you upload.</p>\n\n<p>Is there a way to check if the uploaded file is an image apart from checking the file extension using PHP?</p>\n", Id=1, Tags='php image-processing file-upload upload mime-types', Title='How to check if an uploaded file is an image without mime type?', oneTag='php')

In [9]:
regexTokenizer = RegexTokenizer(inputCol='Body', outputCol='words', pattern="\\W")
df = regexTokenizer.transform(df)
df.head()

Row(Body="<p>I'd like to check if an uploaded file is an image file (e.g png, jpg, jpeg, gif, bmp) or another file. The problem is that I'm using Uploadify to upload the files, which changes the mime type and gives a 'text/octal' or something as the mime type, no matter which file type you upload.</p>\n\n<p>Is there a way to check if the uploaded file is an image apart from checking the file extension using PHP?</p>\n", Id=1, Tags='php image-processing file-upload upload mime-types', Title='How to check if an uploaded file is an image without mime type?', oneTag='php', words=['p', 'i', 'd', 'like', 'to', 'check', 'if', 'an', 'uploaded', 'file', 'is', 'an', 'image', 'file', 'e', 'g', 'png', 'jpg', 'jpeg', 'gif', 'bmp', 'or', 'another', 'file', 'the', 'problem', 'is', 'that', 'i', 'm', 'using', 'uploadify', 'to', 'upload', 'the', 'files', 'which', 'changes', 'the', 'mime', 'type', 'and', 'gives', 'a', 'text', 'octal', 'or', 'something', 'as', 'the', 'mime', 'type', 'no', 'matter', 'which

In [10]:
body_length = udf(lambda x: len(x), IntegerType())
df = df.withColumn('BodyLength', body_length('words'))

In [9]:
df.head()

Row(Body="<p>I'd like to check if an uploaded file is an image file (e.g png, jpg, jpeg, gif, bmp) or another file. The problem is that I'm using Uploadify to upload the files, which changes the mime type and gives a 'text/octal' or something as the mime type, no matter which file type you upload.</p>\n\n<p>Is there a way to check if the uploaded file is an image apart from checking the file extension using PHP?</p>\n", Id=1, Tags='php image-processing file-upload upload mime-types', Title='How to check if an uploaded file is an image without mime type?', oneTag='php', words=['p', 'i', 'd', 'like', 'to', 'check', 'if', 'an', 'uploaded', 'file', 'is', 'an', 'image', 'file', 'e', 'g', 'png', 'jpg', 'jpeg', 'gif', 'bmp', 'or', 'another', 'file', 'the', 'problem', 'is', 'that', 'i', 'm', 'using', 'uploadify', 'to', 'upload', 'the', 'files', 'which', 'changes', 'the', 'mime', 'type', 'and', 'gives', 'a', 'text', 'octal', 'or', 'something', 'as', 'the', 'mime', 'type', 'no', 'matter', 'which

In [11]:
number_of_paragraphs = udf(lambda x: len(re.findall('</p>',x)), IntegerType())
number_of_links = udf(lambda x: len(re.findall('</a>',x)), IntegerType())

In [12]:
df = df.withColumn('NumParagraphs', number_of_paragraphs('Body'))
df = df.withColumn('NumLinks', number_of_links('Body'))

In [13]:
df.head()

Row(Body="<p>I'd like to check if an uploaded file is an image file (e.g png, jpg, jpeg, gif, bmp) or another file. The problem is that I'm using Uploadify to upload the files, which changes the mime type and gives a 'text/octal' or something as the mime type, no matter which file type you upload.</p>\n\n<p>Is there a way to check if the uploaded file is an image apart from checking the file extension using PHP?</p>\n", Id=1, Tags='php image-processing file-upload upload mime-types', Title='How to check if an uploaded file is an image without mime type?', oneTag='php', words=['p', 'i', 'd', 'like', 'to', 'check', 'if', 'an', 'uploaded', 'file', 'is', 'an', 'image', 'file', 'e', 'g', 'png', 'jpg', 'jpeg', 'gif', 'bmp', 'or', 'another', 'file', 'the', 'problem', 'is', 'that', 'i', 'm', 'using', 'uploadify', 'to', 'upload', 'the', 'files', 'which', 'changes', 'the', 'mime', 'type', 'and', 'gives', 'a', 'text', 'octal', 'or', 'something', 'as', 'the', 'mime', 'type', 'no', 'matter', 'which

In [14]:
assembler = VectorAssembler(inputCols = ['BodyLength', 'NumParagraphs', 'NumLinks'], outputCol = "NumFeatures")
df = assembler.transform(df)

In [14]:
df.head()

Row(Body="<p>I'd like to check if an uploaded file is an image file (e.g png, jpg, jpeg, gif, bmp) or another file. The problem is that I'm using Uploadify to upload the files, which changes the mime type and gives a 'text/octal' or something as the mime type, no matter which file type you upload.</p>\n\n<p>Is there a way to check if the uploaded file is an image apart from checking the file extension using PHP?</p>\n", Id=1, Tags='php image-processing file-upload upload mime-types', Title='How to check if an uploaded file is an image without mime type?', oneTag='php', words=['p', 'i', 'd', 'like', 'to', 'check', 'if', 'an', 'uploaded', 'file', 'is', 'an', 'image', 'file', 'e', 'g', 'png', 'jpg', 'jpeg', 'gif', 'bmp', 'or', 'another', 'file', 'the', 'problem', 'is', 'that', 'i', 'm', 'using', 'uploadify', 'to', 'upload', 'the', 'files', 'which', 'changes', 'the', 'mime', 'type', 'and', 'gives', 'a', 'text', 'octal', 'or', 'something', 'as', 'the', 'mime', 'type', 'no', 'matter', 'which

In [15]:
scaler = Normalizer(inputCol = 'NumFeatures', outputCol ='ScaledNumFeatures' )
df = scaler.transform(df)

In [16]:
df.head(2)

[Row(Body="<p>I'd like to check if an uploaded file is an image file (e.g png, jpg, jpeg, gif, bmp) or another file. The problem is that I'm using Uploadify to upload the files, which changes the mime type and gives a 'text/octal' or something as the mime type, no matter which file type you upload.</p>\n\n<p>Is there a way to check if the uploaded file is an image apart from checking the file extension using PHP?</p>\n", Id=1, Tags='php image-processing file-upload upload mime-types', Title='How to check if an uploaded file is an image without mime type?', oneTag='php', words=['p', 'i', 'd', 'like', 'to', 'check', 'if', 'an', 'uploaded', 'file', 'is', 'an', 'image', 'file', 'e', 'g', 'png', 'jpg', 'jpeg', 'gif', 'bmp', 'or', 'another', 'file', 'the', 'problem', 'is', 'that', 'i', 'm', 'using', 'uploadify', 'to', 'upload', 'the', 'files', 'which', 'changes', 'the', 'mime', 'type', 'and', 'gives', 'a', 'text', 'octal', 'or', 'something', 'as', 'the', 'mime', 'type', 'no', 'matter', 'whic

In [16]:
scaler2 = StandardScaler(inputCol = 'NumFeatures', outputCol ='ScaledNumFeatures2', withStd = True)
scalerModel = scaler2.fit(df)
df = scalerModel.transform(df)

In [18]:
df.head(2)

[Row(Body="<p>I'd like to check if an uploaded file is an image file (e.g png, jpg, jpeg, gif, bmp) or another file. The problem is that I'm using Uploadify to upload the files, which changes the mime type and gives a 'text/octal' or something as the mime type, no matter which file type you upload.</p>\n\n<p>Is there a way to check if the uploaded file is an image apart from checking the file extension using PHP?</p>\n", Id=1, Tags='php image-processing file-upload upload mime-types', Title='How to check if an uploaded file is an image without mime type?', oneTag='php', words=['p', 'i', 'd', 'like', 'to', 'check', 'if', 'an', 'uploaded', 'file', 'is', 'an', 'image', 'file', 'e', 'g', 'png', 'jpg', 'jpeg', 'gif', 'bmp', 'or', 'another', 'file', 'the', 'problem', 'is', 'that', 'i', 'm', 'using', 'uploadify', 'to', 'upload', 'the', 'files', 'which', 'changes', 'the', 'mime', 'type', 'and', 'gives', 'a', 'text', 'octal', 'or', 'something', 'as', 'the', 'mime', 'type', 'no', 'matter', 'whic

# Text Preprocessing

In [17]:
cv = CountVectorizer(inputCol='words', outputCol='TF', vocabSize = 1000)
cvmodel = cv.fit(df)
df = cvmodel.transform(df)
df.take(2)

[Row(Body="<p>I'd like to check if an uploaded file is an image file (e.g png, jpg, jpeg, gif, bmp) or another file. The problem is that I'm using Uploadify to upload the files, which changes the mime type and gives a 'text/octal' or something as the mime type, no matter which file type you upload.</p>\n\n<p>Is there a way to check if the uploaded file is an image apart from checking the file extension using PHP?</p>\n", Id=1, Tags='php image-processing file-upload upload mime-types', Title='How to check if an uploaded file is an image without mime type?', oneTag='php', words=['p', 'i', 'd', 'like', 'to', 'check', 'if', 'an', 'uploaded', 'file', 'is', 'an', 'image', 'file', 'e', 'g', 'png', 'jpg', 'jpeg', 'gif', 'bmp', 'or', 'another', 'file', 'the', 'problem', 'is', 'that', 'i', 'm', 'using', 'uploadify', 'to', 'upload', 'the', 'files', 'which', 'changes', 'the', 'mime', 'type', 'and', 'gives', 'a', 'text', 'octal', 'or', 'something', 'as', 'the', 'mime', 'type', 'no', 'matter', 'whic

In [18]:
cvmodel.vocabulary

['p',
 'the',
 'i',
 'to',
 'code',
 'a',
 'gt',
 'lt',
 'is',
 'and',
 'pre',
 'in',
 'this',
 'of',
 'it',
 'that',
 'for',
 '0',
 '1',
 'have',
 'my',
 'if',
 'on',
 'but',
 'with',
 'can',
 'not',
 'be',
 'as',
 't',
 'li',
 'from',
 '2',
 's',
 'http',
 'an',
 'm',
 'strong',
 'new',
 'how',
 'do',
 'com',
 'so',
 'or',
 'at',
 'using',
 'when',
 'am',
 'like',
 'class',
 'id',
 'there',
 'get',
 'are',
 'name',
 'what',
 'any',
 'file',
 'string',
 'data',
 'all',
 'which',
 'want',
 'would',
 'amp',
 'use',
 'java',
 'function',
 'public',
 'some',
 '3',
 'text',
 'error',
 'android',
 'value',
 'c',
 'x',
 'href',
 'you',
 'one',
 'by',
 'user',
 'me',
 'server',
 'type',
 'here',
 'way',
 'return',
 'int',
 'will',
 'div',
 'need',
 'then',
 'set',
 'e',
 'system',
 'has',
 'problem',
 'out',
 'php',
 'no',
 'just',
 '4',
 'org',
 'know',
 'html',
 'only',
 'where',
 'page',
 'application',
 '5',
 'thanks',
 'var',
 'br',
 'we',
 'd',
 'should',
 'does',
 'add',
 'n',
 'true',

In [19]:
# seeing the least common words
cvmodel.vocabulary[::-1] # the list invertes

['wordpress',
 'blog',
 'comes',
 '000',
 'math',
 'master',
 'previous',
 'buttons',
 'desktop',
 'customer',
 'servers',
 '28',
 'shared',
 'existing',
 'canvas',
 'runtime',
 'printf',
 'checkbox',
 'mac',
 'flash',
 'render',
 'thinking',
 'book',
 'article',
 'examples',
 'textbox',
 'syntax',
 'docs',
 'expression',
 'separate',
 'seconds',
 'matrix',
 'dim',
 'difference',
 'job',
 'performance',
 'posts',
 'min',
 'panel',
 'schema',
 'directly',
 'exactly',
 'sum',
 'complete',
 'controls',
 'duplicate',
 'external',
 'exist',
 'sent',
 'attr',
 'h1',
 'native',
 'tool',
 'step',
 'applications',
 'doc',
 'characters',
 'company',
 '64',
 'properly',
 'limit',
 'seen',
 'follows',
 'runs',
 'temp',
 'firefox',
 'issues',
 'makes',
 'common',
 'share',
 'report',
 'hard',
 'displayed',
 'takes',
 'except',
 'params',
 'bind',
 'maps',
 'unable',
 'max',
 'calling',
 'runat',
 'goes',
 'yet',
 'drive',
 'achieve',
 'active',
 'implementation',
 'cursor',
 'records',
 'valid',
 '

In [20]:
cvmodel.vocabulary[-10:]

['customer',
 'desktop',
 'buttons',
 'previous',
 'master',
 'math',
 '000',
 'comes',
 'blog',
 'wordpress']

In [21]:
idf = IDF(inputCol = 'TF', outputCol = 'TFIDF')
idfmodel = idf.fit(df)
df = idfmodel.transform(df)
df.head()

Row(Body="<p>I'd like to check if an uploaded file is an image file (e.g png, jpg, jpeg, gif, bmp) or another file. The problem is that I'm using Uploadify to upload the files, which changes the mime type and gives a 'text/octal' or something as the mime type, no matter which file type you upload.</p>\n\n<p>Is there a way to check if the uploaded file is an image apart from checking the file extension using PHP?</p>\n", Id=1, Tags='php image-processing file-upload upload mime-types', Title='How to check if an uploaded file is an image without mime type?', oneTag='php', words=['p', 'i', 'd', 'like', 'to', 'check', 'if', 'an', 'uploaded', 'file', 'is', 'an', 'image', 'file', 'e', 'g', 'png', 'jpg', 'jpeg', 'gif', 'bmp', 'or', 'another', 'file', 'the', 'problem', 'is', 'that', 'i', 'm', 'using', 'uploadify', 'to', 'upload', 'the', 'files', 'which', 'changes', 'the', 'mime', 'type', 'and', 'gives', 'a', 'text', 'octal', 'or', 'something', 'as', 'the', 'mime', 'type', 'no', 'matter', 'which

In [22]:
indexer = StringIndexer(inputCol = 'oneTag', outputCol = 'label')
df = indexer.fit(df).transform(df)

In [25]:
df.head()

Row(Body="<p>I'd like to check if an uploaded file is an image file (e.g png, jpg, jpeg, gif, bmp) or another file. The problem is that I'm using Uploadify to upload the files, which changes the mime type and gives a 'text/octal' or something as the mime type, no matter which file type you upload.</p>\n\n<p>Is there a way to check if the uploaded file is an image apart from checking the file extension using PHP?</p>\n", Id=1, Tags='php image-processing file-upload upload mime-types', Title='How to check if an uploaded file is an image without mime type?', oneTag='php', words=['p', 'i', 'd', 'like', 'to', 'check', 'if', 'an', 'uploaded', 'file', 'is', 'an', 'image', 'file', 'e', 'g', 'png', 'jpg', 'jpeg', 'gif', 'bmp', 'or', 'another', 'file', 'the', 'problem', 'is', 'that', 'i', 'm', 'using', 'uploadify', 'to', 'upload', 'the', 'files', 'which', 'changes', 'the', 'mime', 'type', 'and', 'gives', 'a', 'text', 'octal', 'or', 'something', 'as', 'the', 'mime', 'type', 'no', 'matter', 'which

In [26]:
df.filter(df.Id == '1112').show()

+--------------------+----+--------------------+--------------------+------+--------------------+----------+-------------+--------+--------------+--------------------+--------------------+--------------------+--------------------+-----+
|                Body|  Id|                Tags|               Title|oneTag|               words|BodyLength|NumParagraphs|NumLinks|   NumFeatures|   ScaledNumFeatures|  ScaledNumFeatures2|                  TF|               TFIDF|label|
+--------------------+----+--------------------+--------------------+------+--------------------+----------+-------------+--------+--------------+--------------------+--------------------+--------------------+--------------------+-----+
|<p>I submitted my...|1112|iphone app-store ...|iPhone app releas...|iphone|[p, i, submitted,...|        63|            1|       0|[63.0,1.0,0.0]|[0.99987404748359...|[0.32825169441613...|(1000,[0,1,2,3,8,...|(1000,[0,1,2,3,8,...|  7.0|
+--------------------+----+--------------------+----

In [27]:
df.persist()

DataFrame[Body: string, Id: bigint, Tags: string, Title: string, oneTag: string, words: array<string>, BodyLength: int, NumParagraphs: int, NumLinks: int, NumFeatures: vector, ScaledNumFeatures: vector, ScaledNumFeatures2: vector, TF: vector, TFIDF: vector, label: double]

In [23]:
df = df.withColumn('Joined_Body_Title', concat(col('Body'),lit(" "), col('Title')))

In [29]:
df.head()

Row(Body="<p>I'd like to check if an uploaded file is an image file (e.g png, jpg, jpeg, gif, bmp) or another file. The problem is that I'm using Uploadify to upload the files, which changes the mime type and gives a 'text/octal' or something as the mime type, no matter which file type you upload.</p>\n\n<p>Is there a way to check if the uploaded file is an image apart from checking the file extension using PHP?</p>\n", Id=1, Tags='php image-processing file-upload upload mime-types', Title='How to check if an uploaded file is an image without mime type?', oneTag='php', words=['p', 'i', 'd', 'like', 'to', 'check', 'if', 'an', 'uploaded', 'file', 'is', 'an', 'image', 'file', 'e', 'g', 'png', 'jpg', 'jpeg', 'gif', 'bmp', 'or', 'another', 'file', 'the', 'problem', 'is', 'that', 'i', 'm', 'using', 'uploadify', 'to', 'upload', 'the', 'files', 'which', 'changes', 'the', 'mime', 'type', 'and', 'gives', 'a', 'text', 'octal', 'or', 'something', 'as', 'the', 'mime', 'type', 'no', 'matter', 'which

In [24]:
regexTokenizer = RegexTokenizer(inputCol='Joined_Body_Title', outputCol='Joined_Body_Title_words', pattern="\\W")
df = regexTokenizer.transform(df)
df.head()

Row(Body="<p>I'd like to check if an uploaded file is an image file (e.g png, jpg, jpeg, gif, bmp) or another file. The problem is that I'm using Uploadify to upload the files, which changes the mime type and gives a 'text/octal' or something as the mime type, no matter which file type you upload.</p>\n\n<p>Is there a way to check if the uploaded file is an image apart from checking the file extension using PHP?</p>\n", Id=1, Tags='php image-processing file-upload upload mime-types', Title='How to check if an uploaded file is an image without mime type?', oneTag='php', words=['p', 'i', 'd', 'like', 'to', 'check', 'if', 'an', 'uploaded', 'file', 'is', 'an', 'image', 'file', 'e', 'g', 'png', 'jpg', 'jpeg', 'gif', 'bmp', 'or', 'another', 'file', 'the', 'problem', 'is', 'that', 'i', 'm', 'using', 'uploadify', 'to', 'upload', 'the', 'files', 'which', 'changes', 'the', 'mime', 'type', 'and', 'gives', 'a', 'text', 'octal', 'or', 'something', 'as', 'the', 'mime', 'type', 'no', 'matter', 'which

In [25]:
body_length = udf(lambda x: len(x), IntegerType())
df = df.withColumn('Length_Body_Title', body_length('Joined_Body_Title_words'))

In [32]:
df.head()

Row(Body="<p>I'd like to check if an uploaded file is an image file (e.g png, jpg, jpeg, gif, bmp) or another file. The problem is that I'm using Uploadify to upload the files, which changes the mime type and gives a 'text/octal' or something as the mime type, no matter which file type you upload.</p>\n\n<p>Is there a way to check if the uploaded file is an image apart from checking the file extension using PHP?</p>\n", Id=1, Tags='php image-processing file-upload upload mime-types', Title='How to check if an uploaded file is an image without mime type?', oneTag='php', words=['p', 'i', 'd', 'like', 'to', 'check', 'if', 'an', 'uploaded', 'file', 'is', 'an', 'image', 'file', 'e', 'g', 'png', 'jpg', 'jpeg', 'gif', 'bmp', 'or', 'another', 'file', 'the', 'problem', 'is', 'that', 'i', 'm', 'using', 'uploadify', 'to', 'upload', 'the', 'files', 'which', 'changes', 'the', 'mime', 'type', 'and', 'gives', 'a', 'text', 'octal', 'or', 'something', 'as', 'the', 'mime', 'type', 'no', 'matter', 'which

In [33]:
df.filter(df.Id == '5123').show()

+--------------------+----+----------+--------------------+------+--------------------+----------+-------------+--------+---------------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+-----------------------+-----------------+
|                Body|  Id|      Tags|               Title|oneTag|               words|BodyLength|NumParagraphs|NumLinks|    NumFeatures|   ScaledNumFeatures|  ScaledNumFeatures2|                  TF|               TFIDF|label|   Joined_Body_Title|Joined_Body_Title_words|Length_Body_Title|
+--------------------+----+----------+--------------------+------+--------------------+----------+-------------+--------+---------------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+-----------------------+-----------------+
|<p>Here's an inte...|5123|git branch|Git branch experi...|   git|[p, here, s, an, ...|       132|            3|       0|[132.0

In [26]:
assembler1 = VectorAssembler(inputCols = ['Length_Body_Title'], outputCol = "NumFeatures_Body_Title")
df = assembler1.transform(df)

In [35]:
df.head()

Row(Body="<p>I'd like to check if an uploaded file is an image file (e.g png, jpg, jpeg, gif, bmp) or another file. The problem is that I'm using Uploadify to upload the files, which changes the mime type and gives a 'text/octal' or something as the mime type, no matter which file type you upload.</p>\n\n<p>Is there a way to check if the uploaded file is an image apart from checking the file extension using PHP?</p>\n", Id=1, Tags='php image-processing file-upload upload mime-types', Title='How to check if an uploaded file is an image without mime type?', oneTag='php', words=['p', 'i', 'd', 'like', 'to', 'check', 'if', 'an', 'uploaded', 'file', 'is', 'an', 'image', 'file', 'e', 'g', 'png', 'jpg', 'jpeg', 'gif', 'bmp', 'or', 'another', 'file', 'the', 'problem', 'is', 'that', 'i', 'm', 'using', 'uploadify', 'to', 'upload', 'the', 'files', 'which', 'changes', 'the', 'mime', 'type', 'and', 'gives', 'a', 'text', 'octal', 'or', 'something', 'as', 'the', 'mime', 'type', 'no', 'matter', 'which

In [27]:
scaler3 = Normalizer(inputCol = 'NumFeatures_Body_Title', outputCol ='ScaledNumFeatures_Body_Title' )
df = scaler3.transform(df)

In [37]:
df.head(2)

[Row(Body="<p>I'd like to check if an uploaded file is an image file (e.g png, jpg, jpeg, gif, bmp) or another file. The problem is that I'm using Uploadify to upload the files, which changes the mime type and gives a 'text/octal' or something as the mime type, no matter which file type you upload.</p>\n\n<p>Is there a way to check if the uploaded file is an image apart from checking the file extension using PHP?</p>\n", Id=1, Tags='php image-processing file-upload upload mime-types', Title='How to check if an uploaded file is an image without mime type?', oneTag='php', words=['p', 'i', 'd', 'like', 'to', 'check', 'if', 'an', 'uploaded', 'file', 'is', 'an', 'image', 'file', 'e', 'g', 'png', 'jpg', 'jpeg', 'gif', 'bmp', 'or', 'another', 'file', 'the', 'problem', 'is', 'that', 'i', 'm', 'using', 'uploadify', 'to', 'upload', 'the', 'files', 'which', 'changes', 'the', 'mime', 'type', 'and', 'gives', 'a', 'text', 'octal', 'or', 'something', 'as', 'the', 'mime', 'type', 'no', 'matter', 'whic

In [38]:
df.filter(df.Id == '512').show()

+--------------------+---+--------------------+--------------------+------+--------------------+----------+-------------+--------+--------------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+-----------------------+-----------------+----------------------+----------------------------+
|                Body| Id|                Tags|               Title|oneTag|               words|BodyLength|NumParagraphs|NumLinks|   NumFeatures|   ScaledNumFeatures|  ScaledNumFeatures2|                  TF|               TFIDF|label|   Joined_Body_Title|Joined_Body_Title_words|Length_Body_Title|NumFeatures_Body_Title|ScaledNumFeatures_Body_Title|
+--------------------+---+--------------------+--------------------+------+--------------------+----------+-------------+--------+--------------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+-----------------------+-----------------+

In [28]:
scaler4 = StandardScaler(inputCol = 'NumFeatures_Body_Title', outputCol ='ScaledNumFeatures_Body_Title3', withStd = True, withMean = True)
scalerModel1 = scaler4.fit(df)
df = scalerModel1.transform(df)

In [40]:
df.head()

Row(Body="<p>I'd like to check if an uploaded file is an image file (e.g png, jpg, jpeg, gif, bmp) or another file. The problem is that I'm using Uploadify to upload the files, which changes the mime type and gives a 'text/octal' or something as the mime type, no matter which file type you upload.</p>\n\n<p>Is there a way to check if the uploaded file is an image apart from checking the file extension using PHP?</p>\n", Id=1, Tags='php image-processing file-upload upload mime-types', Title='How to check if an uploaded file is an image without mime type?', oneTag='php', words=['p', 'i', 'd', 'like', 'to', 'check', 'if', 'an', 'uploaded', 'file', 'is', 'an', 'image', 'file', 'e', 'g', 'png', 'jpg', 'jpeg', 'gif', 'bmp', 'or', 'another', 'file', 'the', 'problem', 'is', 'that', 'i', 'm', 'using', 'uploadify', 'to', 'upload', 'the', 'files', 'which', 'changes', 'the', 'mime', 'type', 'and', 'gives', 'a', 'text', 'octal', 'or', 'something', 'as', 'the', 'mime', 'type', 'no', 'matter', 'which

df.filter(df.Id == '512').show()

In [29]:
minmaxscaler = MinMaxScaler(inputCol = 'NumFeatures_Body_Title', outputCol ='ScaledMinMax')
minmaxscaler_Model = minmaxscaler.fit(df)
df = minmaxscaler_Model.transform(df)

In [42]:
df.filter(df.Id == '512').show()

+--------------------+---+--------------------+--------------------+------+--------------------+----------+-------------+--------+--------------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+-----------------------+-----------------+----------------------+----------------------------+-----------------------------+--------------------+
|                Body| Id|                Tags|               Title|oneTag|               words|BodyLength|NumParagraphs|NumLinks|   NumFeatures|   ScaledNumFeatures|  ScaledNumFeatures2|                  TF|               TFIDF|label|   Joined_Body_Title|Joined_Body_Title_words|Length_Body_Title|NumFeatures_Body_Title|ScaledNumFeatures_Body_Title|ScaledNumFeatures_Body_Title3|        ScaledMinMax|
+--------------------+---+--------------------+--------------------+------+--------------------+----------+-------------+--------+--------------+--------------------+--------------------+---------

In [30]:
pca = PCA(k = 100, inputCol = 'TFIDF', outputCol = 'pcaTFIDF')
model = pca.fit(df)
df = model.transform(df)

In [31]:
df.head()

Row(Body="<p>I'd like to check if an uploaded file is an image file (e.g png, jpg, jpeg, gif, bmp) or another file. The problem is that I'm using Uploadify to upload the files, which changes the mime type and gives a 'text/octal' or something as the mime type, no matter which file type you upload.</p>\n\n<p>Is there a way to check if the uploaded file is an image apart from checking the file extension using PHP?</p>\n", Id=1, Tags='php image-processing file-upload upload mime-types', Title='How to check if an uploaded file is an image without mime type?', oneTag='php', words=['p', 'i', 'd', 'like', 'to', 'check', 'if', 'an', 'uploaded', 'file', 'is', 'an', 'image', 'file', 'e', 'g', 'png', 'jpg', 'jpeg', 'gif', 'bmp', 'or', 'another', 'file', 'the', 'problem', 'is', 'that', 'i', 'm', 'using', 'uploadify', 'to', 'upload', 'the', 'files', 'which', 'changes', 'the', 'mime', 'type', 'and', 'gives', 'a', 'text', 'octal', 'or', 'something', 'as', 'the', 'mime', 'type', 'no', 'matter', 'which